## Lesson 2 Demo 1: Creating Normalized Tables
<p align="center">
<img src="../Docs/postgresql_logo.png" alt="PostgreSQL logo" style="height: 250px;"/>
</p>

In this demo we are going to walk to through the basics of modeling data in normalized form. We will create tables in PostgreSQL, insert rows of data, and do a simple JOIN SQL query to show how these tables can work together.

**Import the library**

In [41]:
import psycopg2

Create a connection to the database, get a cursor, and set autocoomit to true

In [42]:
try:
    conn = psycopg2.connect("dbname=udacity")
except psycopg2.Error as e:
    print("Error: Could not make connection to the Postgres database")
    print(e)

try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print("Error: Could not get curser to the Database")
    print(e)

conn.set_session(autocommit=True)

**Let's imagine we have a table called Music Library.**

> Table name: music_library <li>
> column 0: Album ID <li>
> column 1: Album Name <li>
> column 2: artist Name <li>
> column 3: Year <li>
> column 4: List of songs

<p align="center">
<img src="../Docs/L2_D1_table_1.png" alt="PostgreSQL logo" style="width: 600px;"/>
</p>

Now to transte this information into a Create Table Statement and insert the data

In [43]:
try:
    cur.execute("CREATE TABLE IF NOT EXISTS music_library   (album_id int, \
                                                            album_name varchar, artist_name varchar, \
                                                            year int, songs text[]);")

except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

try:
    cur.execute("INSERT INTO music_library (album_id, album_name, artist_name, year, songs) \
                VALUES (%s, %s, %s, %s, %s)", \
                (1, "Rubber Soul", "The Beatles", 1965, ["Michelle", "Think For Yourself", "In My Life"]))

except psycopg2.Error as e:
    print("Error: Issue inserting rows")
    print(e)

try:
    cur.execute("INSERT INTO music_library (album_id, album_name, artist_name, year, songs) \
                VALUES (%s, %s, %s, %s, %s)", \
                (2, "Let it Be", "The Beatles", 1970, ["Let it Be", "Across The Universe"]))

except psycopg2.Error as e:
    print("Error: Issue inserting rows")
    print(e)

Quering the table music_library

In [44]:
try:
    cur.execute("SELECT * FROM music_library")

except psycopg2.Error as e:
    print("Error: Getting data")
    print(e)

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

(1, 'Rubber Soul', 'The Beatles', 1965, ['Michelle', 'Think For Yourself', 'In My Life'])
(2, 'Let it Be', 'The Beatles', 1970, ['Let it Be', 'Across The Universe'])


**Moving to 1st Normal Form (1NF)**

This data has not been normalized. To get this data into 1st normal form, we will need to remove any collections or list of data. We need to breack up the list of songs into individuals rows.

> Table name: music_library_1NF <li>
> column 0: Album ID <li>
> column 1: Album Name <li>
> column 2: artist Name <li>
> column 3: Year <li>
> column 4: List of songs

<p align="center">
<img src="../Docs/L2_D1_table_2.png" alt="PostgreSQL logo" style="width: 600px;"/>
</p>

Now to transte this information into a Create Table Statement and insert the data

In [45]:
try:
    cur.execute("CREATE TABLE IF NOT EXISTS music_library_1NF   (album_id int, \
                                                            album_name varchar, artist_name varchar, \
                                                            year int, song_name text);")

except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

#list of rows to be inserted
values =       [(1, "Rubber Soul", "The Beatles", 1965, "Michelle"),
                (1, "Rubber Soul", "The Beatles", 1965, "Think For Yourself"),
                (1, "Rubber Soul", "The Beatles", 1965, "In My Life"),
                (2, "Let it Be", "The Beatles", 1970, "Let it Be"),
                (2, "Let it Be", "The Beatles", 1970, "Across The Universe")]

#insert statement to insert rows into table music_library_1NF
sql = "INSERT INTO music_library_1NF (album_id, album_name, artist_name, year, song_name) VALUES (%s, %s, %s, %s, %s)"

try:
    cur.executemany(sql, values)

except psycopg2.Error as e:
    print("Error: Issue inserting rows")
    print(e)

Quering the table music_library_1NF

In [46]:
try:
    cur.execute("SELECT * FROM music_library_1NF")

except psycopg2.Error as e:
    print("Error: Getting data")
    print(e)

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

(1, 'Rubber Soul', 'The Beatles', 1965, 'Michelle')
(1, 'Rubber Soul', 'The Beatles', 1965, 'Think For Yourself')
(1, 'Rubber Soul', 'The Beatles', 1965, 'In My Life')
(2, 'Let it Be', 'The Beatles', 1970, 'Let it Be')
(2, 'Let it Be', 'The Beatles', 1970, 'Across The Universe')


**Moving to 2st Normal Form (1NF)**

We have moved our data to be in 1NF which is the first step in moving to 2nd Normal Form. Our table is not yer in 2nd Normal Form. While each of our records in our table is unique, our Primary Key (album id) is not unique. We need to breack this up into two tables, album library and song library. 

> Table name: album_library_2NF <li>
> column 0: Album ID <li>
> column 1: Album Name <li>
> column 2: Artist Name <li>
> column 3: Year <li>

> Table name: song_library_2NF <li>
> column 0: Song Id <li>
> column 1: Song Name <li>
> column 2: Album Id <li>

<p align="center">
<img src="../Docs/L2_D1_table_3.png" alt="PostgreSQL logo" style="width: 600px;"/>
</p>

Now to transte this information into a Create Table Statement and insert the data

In [47]:
try:
    cur.execute("CREATE TABLE IF NOT EXISTS album_library_2NF   (album_id int, \
                                                            album_name varchar, artist_name varchar, year int);")

except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

try:
    cur.execute("CREATE TABLE IF NOT EXISTS song_library_2NF   (song_id int, \
                                                            song_name varchar, album_id int);")

except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

In [48]:
#list of rows to be inserted
values =       [(1, "Rubber Soul", "The Beatles", 1965),
                (2, "Let it Be", "The Beatles", 1970)]

#insert statement to insert rows into table music_library_1NF
sql = "INSERT INTO album_library_2NF (album_id, album_name, artist_name, year) VALUES (%s, %s, %s, %s)"

try:
    cur.executemany(sql, values)

except psycopg2.Error as e:
    print("Error: Issue inserting rows")
    print(e)

In [49]:
#list of rows to be inserted
values =       [(1,1, "Michelle"),
                (2,1, "Think For Yourself"),
                (3,1, "In My Life"),
                (4,2, "Let it Be"),
                (5,2, "Across The Universe")]

#insert statement to insert rows into table music_library_1NF
sql = "INSERT INTO song_library_2NF (song_id, album_id, song_name) VALUES (%s, %s, %s)"

try:
    cur.executemany(sql, values)

except psycopg2.Error as e:
    print("Error: Issue inserting rows")
    print(e)

In [50]:
print("\nalbum_library_2NF\n")
try:
    cur.execute("SELECT * from album_library_2NF;")
except psycopg2.Error as e:
    print("Error: Issue getting rows")
    print(e)

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

print("\nsong_library_2NF\n")
try:
    cur.execute("SELECT * from song_library_2NF;")
except psycopg2.Error as e:
    print("Error: Issue getting rows")
    print(e)

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()


album_library_2NF

(1, 'Rubber Soul', 'The Beatles', 1965)
(2, 'Let it Be', 'The Beatles', 1970)

song_library_2NF

(1, 'Michelle', 1)
(2, 'Think For Yourself', 1)
(3, 'In My Life', 1)
(4, 'Let it Be', 2)
(5, 'Across The Universe', 2)


Let's do a `JOIN` on this table so we can get all the information we had in our first Table

In [51]:
print("\nJoin to get the first table\n")
try:
    cur.execute("SELECT * FROM album_library_2NF JOIN song_library_2NF ON album_library_2NF.album_id = song_library_2NF.album_id ;")
except psycopg2.Error as e:
    print("Error: getting data")
    print(e)

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()


Join to get the first table

(1, 'Rubber Soul', 'The Beatles', 1965, 1, 'Michelle', 1)
(1, 'Rubber Soul', 'The Beatles', 1965, 2, 'Think For Yourself', 1)
(1, 'Rubber Soul', 'The Beatles', 1965, 3, 'In My Life', 1)
(2, 'Let it Be', 'The Beatles', 1970, 4, 'Let it Be', 2)
(2, 'Let it Be', 'The Beatles', 1970, 5, 'Across The Universe', 2)


*Moving to 3rd Normal Form (3NF)*
Check our table for any transitive dependencies. Album_library can move Artist_name to its own table, called Artist, which will leave us with 3 tables

> Table name: album_library_3NF <li>
> column 0: Album Id <li>
> column 1: Album Name <li>
> column 2: Artist Id <li>
> column 3: Year

> Table name: song_library_3NF <li>
> column 0: Song Id <li>
> column 1: Song Name <li>
> column 2: Album Id

> Table name: artist_library_3NF <li>
> column 0: Artist Id <li>
> column 1: Artist Name

<p align="center">
<img src="../Docs/L2_D1_table_4.png" alt="PostgreSQL logo" style="width: 600px;"/>
</p>

Now to transte this information into a Create Table Statement and insert the data

In [52]:
try:
    cur.execute("CREATE TABLE IF NOT EXISTS album_library_3NF   (album_id int, \
                                                            album_name varchar, artist_id int, year int);")

except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

try:
    cur.execute("CREATE TABLE IF NOT EXISTS song_library_3NF   (song_id int, \
                                                            song_name varchar, album_id int);")

except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

try:
    cur.execute("CREATE TABLE IF NOT EXISTS artist_library_3NF   (artist_id int, \
                                                            artist_name varchar);")

except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

In [53]:
#insert statement to insert rows into table song_library_3NF
sql = "INSERT INTO song_library_3NF (song_id, album_id, song_name) VALUES (%s, %s, %s)"

#list of rows to be inserted
values =       [(1,1, "Michelle"),
                (2,1, "Think For Yourself"),
                (3,1, "In My Life"),
                (4,2, "Let it Be"),
                (5,2, "Across The Universe")]

try:
    cur.executemany(sql, values)

except psycopg2.Error as e:
    print("Error: Issue inserting rows")
    print(e)

#insert statement to insert rows into table album_library_3NF
sql = "INSERT INTO album_library_3NF (album_id, album_name, artist_id, year) VALUES (%s, %s, %s, %s)"

#list of rows to be inserted
values =       [(1, "Rubber Soul", 1, 1965),
                (2, "Let it Be", 1, 1970)]

try:
    cur.executemany(sql, values)

except psycopg2.Error as e:
    print("Error: Issue inserting rows")
    print(e)

#insert statement to insert rows into table artist_library_3NF
sql = "INSERT INTO artist_library_3NF (artist_id, artist_name) VALUES (%s, %s)"

#list of rows to be inserted
values = [(1, "The Beatles")]

try:
    cur.executemany(sql, values)

except psycopg2.Error as e:
    print("Error: Issue inserting rows")
    print(e)

In [54]:
#select statement

print("\nsong_library_3NF\n")
try:
    cur.execute("SELECT * from song_library_3NF;")
except psycopg2.Error as e:
    print("Error: Issue getting rows")
    print(e)

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

print("\nalbum_library_3NF\n")
try:
    cur.execute("SELECT * from album_library_3NF;")
except psycopg2.Error as e:
    print("Error: Issue getting rows")
    print(e)

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

print("\nartist_library_3NF\n")
try:
    cur.execute("SELECT * from artist_library_3NF;")
except psycopg2.Error as e:
    print("Error: Issue getting rows")
    print(e)

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()




song_library_3NF

(1, 'Michelle', 1)
(2, 'Think For Yourself', 1)
(3, 'In My Life', 1)
(4, 'Let it Be', 2)
(5, 'Across The Universe', 2)

album_library_3NF

(1, 'Rubber Soul', 1, 1965)
(2, 'Let it Be', 1, 1970)

artist_library_3NF

(1, 'The Beatles')


Let's do two `JOIN` on these 3 tables so we can get all the information we had in our first Table

In [55]:
print("\nJoin to get the first table\n")
try:
    cur.execute("SELECT album_library_3NF.album_id, album_library_3NF.album_name, artist_library_3NF.artist_name, \
                album_library_3NF.year, song_library_3NF.song_name \
                FROM (artist_library_3NF JOIN album_library_3NF ON \
                artist_library_3NF.artist_id = album_library_3NF.artist_id) JOIN \
                song_library_3NF ON album_library_3NF.album_id = song_library_3NF.album_id;")
except psycopg2.Error as e:
    print("Error: getting data")
    print(e)

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()


Join to get the first table

(1, 'Rubber Soul', 'The Beatles', 1965, 'Michelle')
(1, 'Rubber Soul', 'The Beatles', 1965, 'Think For Yourself')
(1, 'Rubber Soul', 'The Beatles', 1965, 'In My Life')
(2, 'Let it Be', 'The Beatles', 1970, 'Let it Be')
(2, 'Let it Be', 'The Beatles', 1970, 'Across The Universe')


## DONE! We have Normnalized our dataset!

### For the sake of the demo, I'll drop the tables

In [56]:
try:
    cur.execute("DROP TABLE music_library; DROP TABLE music_library_1NF; DROP TABLE album_library_2NF; DROP TABLE song_library_2NF; \
                DROP TABLE album_library_3NF; DROP TABLE song_library_3NF; DROP TABLE artist_library_3NF;")
except psycopg2.Error as e:
    print("Error: Dropping table")
    print(e)

### And finally close your cursor and connection.

In [57]:
cur.close()
conn.close()